In [1]:
# imports here
import numpy as np # to read the bin files
import open3d as o3d # for working with 3d data
import matplotlib.pyplot as plt


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Loading the file

In [5]:
# Load the .bin File

def load_kitti_bin(file_path):
    data = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
    return data
file_no = "000010"
file_path = "data/velodyne/" + file_no + ".bin"
points = load_kitti_bin(file_path)
print(f"Loaded {points.shape[0]} points from {file_path}")

Loaded 115875 points from data/velodyne/000010.bin


## Visualizing the Point Cloud

In [12]:
def convert_to_open3d_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    if points.shape[1] == 4:
        reflectance = points[:, 3]
        reflectance_normalized = (reflectance - reflectance.min()) / (reflectance.max() - reflectance.min())
        colormap = plt.cm.gray
        colors = colormap(reflectance_normalized)[:, :3]
        pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

pcd = convert_to_open3d_point_cloud(points)
print("Point cloud converted to Open3D format.")

Point cloud converted to Open3D format.


In [13]:
# visualize
o3d.visualization.draw_geometries([pcd])

## Optional operations

In [48]:
# Downsample
voxel_size = 0.1
pcd_downsampled = pcd.voxel_down_sample(voxel_size=voxel_size)

# Remove noise
cl, ind = pcd_downsampled.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_filtered = pcd_downsampled.select_by_index(ind)

In [49]:
# Visualize
o3d.visualization.draw_geometries([pcd_filtered])

In [ ]:
# Save to file
o3d.io.write_point_cloud("processed_kitti.ply", pcd_filtered)
print("Processed point cloud saved as 'processed_kitti.ply'.")

## Overlaying Bounding Boxes

In [14]:
def load_labels(label_file):
    bounding_boxes = []
    with open(label_file, "r") as f:
        for line in f.readlines():
            data = line.strip().split()
            bbox = {
                "class": data[0],
                "dimensions": [float(data[8]), float(data[9]), float(data[10])],  # h, w, l
                "location": [float(data[11]), float(data[12]), float(data[13])],  # x, y, z
                "rotation_y": float(data[14])  # ry
            }
            bounding_boxes.append(bbox)
    return bounding_boxes

label_file = f"data/label/{file_no}.txt"
bounding_boxes = load_labels(label_file)

In [19]:
def create_3d_bounding_box(bbox):
    import math

    class_colors = {
    "Car": [1, 0, 0],        # Red
    "Pedestrian": [0, 1, 0], # Green
    "Cyclist": [0, 0, 1]     # Blue
    }

    # Extract parameters
    h, w, l = bbox["dimensions"]
    x, y, z = bbox["location"]
    ry = bbox["rotation_y"]

    # Define the 8 corners of the bounding box (before rotation)
    corners = np.array([
        [l/2, 0,  w/2], [l/2, 0, -w/2], [-l/2, 0, -w/2], [-l/2, 0,  w/2],  # Bottom face
        [l/2, h,  w/2], [l/2, h, -w/2], [-l/2, h, -w/2], [-l/2, h,  w/2]   # Top face
    ])

    # Rotate and translate the corners
    rotation_matrix = np.array([
        [math.cos(ry), 0, math.sin(ry)],
        [0, 1, 0],
        [-math.sin(ry), 0, math.cos(ry)]
    ])
    corners = np.dot(corners, rotation_matrix.T) + np.array([x, y, z])

    # Define the edges of the bounding box
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical edges
    ]

    # Create an Open3D LineSet
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(corners)
    line_set.lines = o3d.utility.Vector2iVector(edges)
    line_set.colors = o3d.utility.Vector3dVector([class_colors.get(bbox["class"], [1, 1, 1])] * len(edges))

    return line_set

# Create 3D bounding boxes
line_sets = [create_3d_bounding_box(bbox) for bbox in bounding_boxes]

In [20]:
# Visualize point cloud with bounding boxes
o3d.visualization.draw_geometries([pcd, *line_sets])

## Fixing alignment of bounding boxes with pointclouds with calibration matrix

In [23]:
def load_calibration(calib_file):
    calib = {}
    with open(calib_file, "r") as f:
        for line in f.readlines():
            if not line or ":" not in line:
                continue  # Skip blank or incorrectly formatted lines
            key, value = line.split(":", 1)
            value = [float(x) for x in value.strip().split()]
            if key.startswith("P"):
                calib[key] = np.array(value).reshape(3, 4)  # 3x4 projection matrices
            elif key == "R0_rect":
                calib[key] = np.array(value).reshape(3, 3)  # 3x3 rectification matrix
            elif key == "Tr_velo_to_cam":
                calib[key] = np.array(value).reshape(3, 4)  # 3x4 transformation matrix
            else:
                calib[key] = np.array(value)

    # Compute Tr_velo_to_rect
    R0_rect = np.eye(4)
    R0_rect[:3, :3] = calib["R0_rect"]
    Tr_velo_to_cam = np.vstack((calib["Tr_velo_to_cam"], [0, 0, 0, 1]))
    calib["Tr_velo_to_rect"] = np.dot(R0_rect, Tr_velo_to_cam)
    return calib


# Loading calib file
calib_file = f"data\calib\{file_no}.txt"
calibration = load_calibration(calib_file)


In [24]:
def transform_bbox_to_lidar(bbox, calib):
    # Extract bounding box center in the camera frame
    center_camera = np.array(bbox["location"] + [1])  # Homogeneous coordinates
    # Transform to LiDAR frame
    Tr_rect_to_velo = np.linalg.inv(calib["Tr_velo_to_rect"])
    center_lidar = np.dot(Tr_rect_to_velo, center_camera)[:3]
    bbox["location"] = center_lidar.tolist()
    return bbox

# Apply transformation
bounding_boxes_lidar = [transform_bbox_to_lidar(bbox, calibration) for bbox in bounding_boxes]


In [25]:
for bbox in bounding_boxes_lidar:
    bbox["location"][2] -= bbox["dimensions"][0] / 2  # Subtract half the height


In [26]:
# # Load point cloud
# # pcd = convert_to_open3d_point_cloud(points)

# # Transform bounding boxes to LiDAR frame
# bounding_boxes_lidar = [transform_bbox_to_lidar(bbox, calibration) for bbox in bounding_boxes]

# # Adjust heights for KITTI
# for bbox in bounding_boxes_lidar:
#     bbox["location"][2] -= bbox["dimensions"][0] / 2

# Create 3D bounding boxes
line_sets = [create_3d_bounding_box(bbox) for bbox in bounding_boxes_lidar]

# Visualize point cloud with bounding boxes
o3d.visualization.draw_geometries(
    [pcd, *line_sets],
)
